# SU(d)  Dicke state $|D^{n}(\vec{k})\rangle$ preparation: MPS

Cirq implementation of a circuit in:

"Simple ways of preparing qudit Dicke states", https://arxiv.org/abs/2507.13308

by Noah B. Kerzner, Federico Galeazzi and Rafael I. Nepomechie

Code written by Noah Kerzner

- to run code, in cell use `qudit_dicke(k,mode)`
- set mode to 1 to print circuit
- basis states output of form $|${working qudits}{mps ancilary qudit}{extra qubit}$\rangle$

# Added Gates 

- `XijGate(d,i,j)`
  - General not gate for qudit of dimension d notted from basis i into basis j
- `ditRotation(theta,i,j,d)`
    - Rotates qudit of dimensions d in bloch sphere from basis i into basis j by an angle $\theta$

In [ ]:
pip install cirq

In [13]:
import cirq
import numpy as np
import itertools
from scipy.special import factorial


class XijGate(cirq.Gate):
    def __init__(self, d: int, i: int, j: int):
        assert 0 <= i < d and 0 <= j < d and i != j, "i and j must be distinct and in range"
        self.d = d
        self.i = i
        self.j = j

    def _qid_shape_(self):
        return (self.d,)

    def _unitary_(self):
        mat = np.eye(self.d, dtype=complex)
        mat[self.j, self.i] = 1  # Set transition i -> j
        mat[self.i, self.i] = 0  # Zero out original diagonal
        return mat

    def _circuit_diagram_info_(self, args):
        return f"X^{self.i}->{self.j}"



class ditrotation(cirq.Gate):
    def __init__(self, theta, i, j, d):
        if i == j or not (0 <= i < d) or not (0 <= j < d):
            raise ValueError("Indices i and j must be different and in range [0, d-1]")
        super().__init__()
        self.theta = theta
        self.i = i
        self.j = j
        self.d = d

    def _qid_shape_(self):
        return (self.d,)

    def _unitary_(self):
        U = np.eye(self.d, dtype=np.complex128)
        c, s = np.cos(self.theta / 2), np.sin(self.theta / 2)
        U[self.i, self.i] = c
        U[self.j, self.j] = c
        U[self.i, self.j] = -s
        U[self.j, self.i] = s
        return U

    def _circuit_diagram_info_(self, args):
        return f"R{self.i}{self.j}({self.theta:.2f})"




# Angle and $A^i(\vec k)$ functions

- `a_sets`
  - Class which stores all level sets $A^i(\vec k)$ and handles the lableling functions.
  - Also computes and returns angles for I gate
  - all other non class functions are helper functions for the two tasks above 
  

In [14]:
class a_sets:



    def __init__(self,k_vec,n):
        self.k_vec=k_vec
        self.d=len(k_vec)
        self.n=n
        self._j_forw, self._j_rev, self.chi = make_a_lists(k_vec,n )
        


    def j_forward(self,l:int,set: np.ndarray ) -> int:

        return self._j_forw[l][tuple(set)]
    
    def j_reverse(self,l:int,label: int) -> np.ndarray:

        return self._j_rev[l][label]
    
    def gamma(self, i, p, m):
        if p in self._j_rev[i - 1]:
            a = self.j_reverse(i - 1, p)
            hat_m = m_hat(m, self.d)
            a_new = a + hat_m
            if tuple(a_new) in self._j_forw[i]:
                # If we've already used symbol m to its maximum allowed count, skip
                gamma_val = self._c(self.n, self.k_vec, i, a_new) * self._c(i, a_new, i - 1, a) / self._c(self.n, self.k_vec, i - 1, a)
                return gamma_val
        return 0
        

    def _c(self,n:int,k: np.ndarray,l: int, a:  np.ndarray)-> float:
    
        val = multinomial(l,a) * multinomial(n - l, k - a) / multinomial(n, k)
        return np.sqrt(val)



        

    def get_dit_thetas(self, i, p):
        thetas = []

        gamms=[]
        for m in range(self.d - 1):
            gamma_val = self.gamma(i, p, m)

            gamms.append(gamma_val)
            if gamma_val == 0:
                thetas.append(np.pi)
                continue

            denom = list_prod(thetas)
            
            if denom < 1e-10:
                thetas.append(0)
                continue

            val = gamma_val / denom
            val = np.clip(val, -1, 1)
            theta = 2 * np.arccos(val)

            thetas.append(theta)

        #print(f'gamms/thetas for i= {i}, p= {p}\n{gamms}\n{thetas}\n')
        return thetas



def multinomial(n: int, k: np.ndarray) -> float:
    """
    Computes the multinomial coefficient:
    multinomial(n; k_1, k_2, ..., k_r) = n! / (k_1! * k_2! * ... * k_r!)

    Args:
        n (int): total number of elements (sum of k)
        k (np.ndarray): array of counts in each category

    Returns:
        float: multinomial coefficient
    """
    return factorial(n) / np.prod(factorial(k))


    


def generate_a_l_k(k_vec, l):
    """
    Generates all vectors a = (a_0, ..., a_{d-1}) such that:
    - 0 <= a_i <= k_i for each i
    - sum(a) == l
    """
    ranges = [range(k_i + 1) for k_i in k_vec]
    all_candidates = itertools.product(*ranges)
    j_forward={}
    j_reverse={}
    label=0
    for a in reversed(list(all_candidates)):
        if sum(a)==l:
            valid_vec=np.array(a)
            j_forward[tuple(valid_vec)]=label
            j_reverse[label]=valid_vec
            label+=1
        

    return j_forward, j_reverse



def make_a_lists(k_vec,n):

    forward_j_list=[]
    reverse_j_list=[]

    max_len=0
    for i in range(n+1):
        j_for,j_rev=generate_a_l_k(k_vec,i)
        forward_j_list.append(j_for)
        reverse_j_list.append(j_rev)
        if i ==np.floor(n/2):
            max_len=len(list(j_for.keys()))

    
    return forward_j_list, reverse_j_list, max_len


def m_hat(m:int,d):
    vec=np.array([0]*d)
    vec[m]=1
    return vec




def list_prod(arr):
    """Helper function for angle calculations 

    Args:
        arr (float): list of already calculated thetas

    Returns:
        float: helper product
    """    
    ret = 1
    for arg in arr:
        ret *= np.sin(arg / 2)
    return ret


# gate operations 

- `ditIgate()`
  - the circuit $I^{(i)}_{\vec a_p}$ as described in the paper
- `ditUgate()`
  - makes the product of I gates
- `ditU()`
  - acts all of the $U_i$
- `qudit_dicke(k,mode=0)`
  - prepares SU(d) dicke state for generic $\vec k$
  - set mode to 1 to print circuit

In [15]:

def ditIgate(quds:cirq.LineQid, anc_qud: cirq.NamedQid, qub_anc: cirq.NamedQubit , sets: a_sets, i: int, a: np.ndarray,):
    """I Gate operation for dicke state creatioin

    Args:
        quds (cirq.LineQid): List of working qudits
        anc_qud (cirq.NamedQid): anciliry qudit
        n (int): number of working qudits
        k (int): number of spin operators
        i (int): the curent working qudit number
        l (int): the current I gate number
        s (float): the spin of the system
    """    

    
    p=sets.j_forward(i-1,a)
    thetas = sets.get_dit_thetas(i,p)

    d= sets.d
    chi=sets.chi

    # if wokring qudit is 0 sift anc dit up by one 
    yield cirq.ControlledGate(cirq.X, num_controls=2, control_values=(0,p,), control_qid_shape=(d,chi))(quds[i - 1], anc_qud,qub_anc)


    #d-1 controled rotations
    for j in range(d-1):
        yield cirq.ControlledGate(
            ditrotation(thetas[j], j, j + 1, d),
        )(qub_anc, quds[i - 1])

    # not gates 
    for j in range(d):
        a_new=a + m_hat(j,d)
        if tuple(a_new) in sets._j_forw[i]: # if new vector exists in set, apply gate 
            p_new=sets.j_forward(i,a_new)
            if p==p_new:
                continue

            yield cirq.ControlledGate(
                XijGate(chi,p,p_new),
                num_controls=2,
                control_values=(j,1,),
                control_qid_shape=(d,2,)
            )(quds[i-1],qub_anc, anc_qud)

    # Not back ancilary qubit if neccesary 
    for j in range(d):
        a_new=a + m_hat(j,d)
        if tuple(a_new) in sets._j_forw[i]:
            p_new=sets.j_forward(i,a_new)
            yield cirq.ControlledGate(cirq.X, num_controls=2, control_values=(j,p_new,), control_qid_shape=(d,chi))(quds[i - 1], anc_qud,qub_anc)




def ditUgate(quds, anc_qud,qub_anc, set: a_sets ,i:int):
    """Indiviudal U of i. Code breaks U initary into simpler products of I gates for qudit extension

    Args:
        quds (cirq.LineQid): List of working qudits
        anc_qud (cirq.NamedQid): anciliry qudit
        n (int): number of working qudits
        k (int): number of spin operators
        i (int): the curent working qudit number
        s (float): the spin of the system

    """  

    cur=set._j_rev[i-1]
    for a in list(cur.values()):
        yield ditIgate(quds, anc_qud,qub_anc,set, i, a)

   
    

def ditU(qubs, qud, qub_anc, set:a_sets):
    """
    Full unitary U to run U gates on each dit 

    Args:
        quds (cirq.LineQid): List of working qudits
        anc_qud (cirq.NamedQid): anciliry qudit
        n (int): number of working qudits
        k (int): number of spin operators
        s (float): the spin of the system

    """   
    for i in range(1, set.n+ 1):
        yield ditUgate(qubs, qud,qub_anc, set,i)




def qudit_dicke(k: tuple,mode=0):


    sim = cirq.Simulator()

    n=int(sum(np.array(k)))


    
    set=a_sets(k,n)

    #print(set.gamma(1,0,1))
    #print(tuple(np.ndarray([0,0,1])+m_hat(2,3)) in set._j_forw[2])
    quds = cirq.LineQid.range(n, dimension=len(k))
    qudit = cirq.NamedQid('a', dimension=set.chi)
    qubit=cirq.NamedQubit('b')


    circuit = cirq.Circuit(ditU(quds, qudit,qubit, set))


    if mode==1:
        print(circuit)
    print("\n=== Final State Vector ===")
    result = sim.simulate(circuit)
    #print(circuit)
    print(f'bond dimension = {set.chi}')
    print(cirq.dirac_notation(result.final_state_vector, qid_shape=(len(k),) * n + (set.chi,)+ (2,)))


# Examples

The Final State Vector is expressed in  Dirac notation, and is given by 

$$ |D^{n}(\vec{k})\rangle |\underline{0}\rangle |0\rangle \,, $$ 

where $|\underline{0}\rangle$ is the state of the MPS qudit ancilla, and $|0\rangle$ is the state of the qubit ancilla.

In [16]:
# example with n=3, d=3

k=(1,1,1)

qudit_dicke(k,0)  # mode 0 does not print the circuit

# output vector is of form |{working qubits}{ancilary qudit}{ancilary qubit}>


=== Final State Vector ===
bond dimension = 3
0.41|01200⟩ + 0.41|02100⟩ + 0.41|10200⟩ + 0.41|12000⟩ + 0.41|20100⟩ + 0.41|21000⟩


In [17]:
# example with n=4, d=3

k=(1,1,2)

qudit_dicke(k,0)  # mode 0 does not print the circuit


=== Final State Vector ===
bond dimension = 4
0.29|012200⟩ + 0.29|021200⟩ + 0.29|022100⟩ + 0.29|102200⟩ + 0.29|120200⟩ + 0.29|122000⟩ + 0.29|201200⟩ + 0.29|202100⟩ + 0.29|210200⟩ + 0.29|212000⟩ + 0.29|220100⟩ + 0.29|221000⟩


In [18]:
# example with n=6, d=4
k=(2,1,2,1)
qudit_dicke(k, 0)


=== Final State Vector ===
bond dimension = 10
0.07|0,0,1,2,2,3,0,0⟩ + 0.07|0,0,1,2,3,2,0,0⟩ + 0.07|0,0,1,3,2,2,0,0⟩ + 0.07|0,0,2,1,2,3,0,0⟩ + 0.07|0,0,2,1,3,2,0,0⟩ + 0.07|0,0,2,2,1,3,0,0⟩ + 0.07|0,0,2,2,3,1,0,0⟩ + 0.07|0,0,2,3,1,2,0,0⟩ + 0.07|0,0,2,3,2,1,0,0⟩ + 0.07|0,0,3,1,2,2,0,0⟩ + 0.07|0,0,3,2,1,2,0,0⟩ + 0.07|0,0,3,2,2,1,0,0⟩ + 0.07|0,1,0,2,2,3,0,0⟩ + 0.07|0,1,0,2,3,2,0,0⟩ + 0.07|0,1,0,3,2,2,0,0⟩ + 0.07|0,1,2,0,2,3,0,0⟩ + 0.07|0,1,2,0,3,2,0,0⟩ + 0.07|0,1,2,2,0,3,0,0⟩ + 0.07|0,1,2,2,3,0,0,0⟩ + 0.07|0,1,2,3,0,2,0,0⟩ + 0.07|0,1,2,3,2,0,0,0⟩ + 0.07|0,1,3,0,2,2,0,0⟩ + 0.07|0,1,3,2,0,2,0,0⟩ + 0.07|0,1,3,2,2,0,0,0⟩ + 0.07|0,2,0,1,2,3,0,0⟩ + 0.07|0,2,0,1,3,2,0,0⟩ + 0.07|0,2,0,2,1,3,0,0⟩ + 0.07|0,2,0,2,3,1,0,0⟩ + 0.07|0,2,0,3,1,2,0,0⟩ + 0.07|0,2,0,3,2,1,0,0⟩ + 0.07|0,2,1,0,2,3,0,0⟩ + 0.07|0,2,1,0,3,2,0,0⟩ + 0.07|0,2,1,2,0,3,0,0⟩ + 0.07|0,2,1,2,3,0,0,0⟩ + 0.07|0,2,1,3,0,2,0,0⟩ + 0.07|0,2,1,3,2,0,0,0⟩ + 0.07|0,2,2,0,1,3,0,0⟩ + 0.07|0,2,2,0,3,1,0,0⟩ + 0.07|0,2,2,1,0,3,0,0⟩ + 0.07|0,2,2,1,3,0